Rishabh Johri

rjohri@deloitte.com

# Part 3: Spark SQL & DataFrames
This notebook implements Part 3 of the graded assessment:
- Load CSVs (customers, orders, products)
- Run Spark SQL queries (total spend > X, monthly trends, top-selling category)
- Save results as Parquet and JSON


In [1]:
!pip -q install pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("RetailAnalytics-Part3") \
    .getOrCreate()

spark


In [2]:
customers_path = "/content/customers.csv"
orders_path    = "/content/orders.csv"
products_path  = "/content/products.csv"

customers_df = spark.read.csv(customers_path, header=True, inferSchema=True)
orders_df    = spark.read.csv(orders_path,    header=True, inferSchema=True)
products_df  = spark.read.csv(products_path,  header=True, inferSchema=True)

print("Customers columns:", customers_df.columns)
print("Orders columns:", orders_df.columns)
print("Products columns:", products_df.columns)


Customers columns: ['customer_id', 'name', 'country', 'age', 'gender']
Orders columns: ['order_id', 'customer_id', 'product', 'category', 'quantity', 'price', 'order_date']
Products columns: ['product', 'category', 'price']


In [3]:
def first_present(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

ord_qty_col = first_present(orders_df, ["quantity", "qty", "Quantity"])
ord_unit_price_col = first_present(orders_df, ["unit_price", "UnitPrice", "unitprice", "price_per_unit"])
ord_total_amt_col = first_present(orders_df, ["total_amount", "TotalAmount", "amount", "Amount", "order_amount"])
ord_price_col = first_present(orders_df, ["price", "Price", "unit_price", "UnitPrice"])
ord_date_col = first_present(orders_df, ["order_date", "OrderDate", "date", "Date", "order_datetime", "timestamp"])

if ord_date_col is None:
    raise ValueError("Could not find an order date column in orders file.")

orders_with_date = orders_df.withColumn(
    "_order_ts",
    F.to_timestamp(F.col(ord_date_col))
).withColumn(
    "order_date",
    F.to_date(F.col("_order_ts"))
).drop("_order_ts")

if ord_qty_col and ord_unit_price_col:
    orders_clean = orders_with_date.withColumn(
        "revenue",
        F.col(ord_qty_col).cast("double") * F.col(ord_unit_price_col).cast("double")
    )
elif ord_total_amt_col:
    orders_clean = orders_with_date.withColumn(
        "revenue",
        F.col(ord_total_amt_col).cast("double")
    )
elif ord_qty_col and ord_price_col:
    orders_clean = orders_with_date.withColumn(
        "revenue",
        F.col(ord_qty_col).cast("double") * F.col(ord_price_col).cast("double")
    )
else:
    raise ValueError("Could not derive revenue.")

orders_clean.select("order_date","revenue").show(5, truncate=False)


+----------+-------+
|order_date|revenue|
+----------+-------+
|2023-01-01|799.96 |
|2023-01-02|29.97  |
|2023-01-03|29.98  |
|2023-01-04|19.98  |
|2023-01-05|44.97  |
+----------+-------+
only showing top 5 rows



In [4]:
customers_df.createOrReplaceTempView("customers")
orders_clean.createOrReplaceTempView("orders")
products_df.createOrReplaceTempView("products")


In [7]:
# figure out join keys
cust_key = None
for c in ["customer_id", "cust_id", "CustomerID", "Customer_Id"]:
    if c in customers_df.columns:
        cust_key = c
        break

ord_cust_key = None
for c in ["customer_id", "cust_id", "CustomerID", "Customer_Id"]:
    if c in orders_clean.columns:
        ord_cust_key = c
        break

if not cust_key or not ord_cust_key:
    raise ValueError("Couldn't find a common customer key. Expected columns like customer_id/cust_id in both customers and orders.")

# Make a view that standardizes the join key name
customers_std = customers_df.withColumnRenamed(cust_key, "customer_id_std")
orders_std = orders_clean.withColumnRenamed(ord_cust_key, "customer_id_std")
customers_std.createOrReplaceTempView("customers_std")
orders_std.createOrReplaceTempView("orders_std")

# Build SELECT and GROUP BY with explicit customer columns (no "c.*")
cust_cols = customers_std.columns  # e.g., ["customer_id_std","name","country","age","gender"]
select_cols = ", ".join([f"c.`{c}`" for c in cust_cols])
group_by_cols = ", ".join([f"c.`{c}`" for c in cust_cols])

X = 500.0  # adjust your threshold

sql_customers_over_X = f"""
SELECT
  {select_cols},
  ROUND(SUM(o.revenue), 2) AS total_spend
FROM customers_std c
JOIN orders_std o
  ON c.customer_id_std = o.customer_id_std
GROUP BY {group_by_cols}
HAVING SUM(o.revenue) > {X}
ORDER BY total_spend DESC
"""

customers_over_X_df = spark.sql(sql_customers_over_X)
customers_over_X_df.show(20, truncate=False)


+---------------+----------+-------+---+------+-----------+
|customer_id_std|name      |country|age|gender|total_spend|
+---------------+----------+-------+---+------+-----------+
|7              |Customer_7|USA    |28 |Male  |1084.89    |
|8              |Customer_8|Canada |23 |Male  |629.95     |
+---------------+----------+-------+---+------+-----------+



In [8]:
sql_monthly_trend = """
SELECT
  DATE_FORMAT(order_date, 'yyyy-MM') AS year_month,
  ROUND(SUM(revenue), 2) AS monthly_revenue,
  COUNT(*) AS order_count
FROM orders_std
GROUP BY DATE_FORMAT(order_date, 'yyyy-MM')
ORDER BY year_month
"""

monthly_trend_df = spark.sql(sql_monthly_trend)
monthly_trend_df.show(50, truncate=False)


+----------+---------------+-----------+
|year_month|monthly_revenue|order_count|
+----------+---------------+-----------+
|2023-01   |2554.7         |10         |
+----------+---------------+-----------+



In [11]:
customers_std.createOrReplaceTempView("customers_std")
orders_std.createOrReplaceTempView("orders_std")
products_df.createOrReplaceTempView("products")

# 1) Detect join key candidates present in each table
candidate_keys = ["product_id", "prod_id", "product", "sku", "ProductID", "Product_Id", "Product"]

join_key_orders = next((c for c in candidate_keys if c in orders_std.columns), None)
join_key_products = next((c for c in candidate_keys if c in products_df.columns), None)

if not join_key_orders or not join_key_products:
    raise ValueError(
        f"Couldn't find a common product key. "
        f"Orders has {orders_std.columns}; Products has {products_df.columns}. "
        f"Expected one of {candidate_keys} in both."
    )

# 2) Detect category column in products
cat_candidates = ["category", "Category", "category_name", "CategoryName"]
cat_col = next((c for c in cat_candidates if c in products_df.columns), None)
if not cat_col:
    raise ValueError(f"No category column found in products. Expected one of {cat_candidates}.")

# 3) Standardize column names for a clean join
products_std = (
    products_df
    .withColumnRenamed(join_key_products, "prod_key_std")
    .withColumnRenamed(cat_col, "category_std")
)
orders_w_prod = (
    orders_std
    .withColumnRenamed(join_key_orders, "prod_key_std")
)

products_std.createOrReplaceTempView("products_std")
orders_w_prod.createOrReplaceTempView("orders_w_prod")

# 4) Run SQL using the standardized join key
sql_top_category = """
SELECT
  p.category_std AS category,
  ROUND(SUM(o.revenue), 2) AS total_revenue,
  COUNT(*) AS order_rows
FROM orders_w_prod o
JOIN products_std p
  ON o.prod_key_std = p.prod_key_std
GROUP BY p.category_std
ORDER BY total_revenue DESC
"""

top_category_df = spark.sql(sql_top_category)
top_category_df.show(20, truncate=False)

+-----------+-------------+----------+
|category   |total_revenue|order_rows|
+-----------+-------------+----------+
|Sports     |1234.86      |4         |
|Electronics|599.97       |1         |
|Clothing   |419.96       |2         |
|Books      |269.94       |2         |
|Grocery    |29.97        |1         |
+-----------+-------------+----------+



In [12]:
output_base = "/content/output/retail_analytics"

(
    customers_over_X_df
    .coalesce(1)
    .write.mode("overwrite")
    .parquet(f"{output_base}/customers_over_X_parquet")
)
(
    customers_over_X_df
    .coalesce(1)
    .write.mode("overwrite")
    .json(f"{output_base}/customers_over_X_json")
)

(
    monthly_trend_df
    .write.mode("overwrite")
    .parquet(f"{output_base}/monthly_trend_parquet")
)
(
    monthly_trend_df
    .write.mode("overwrite")
    .json(f"{output_base}/monthly_trend_json")
)

(
    top_category_df
    .write.mode("overwrite")
    .parquet(f"{output_base}/top_category_parquet")
)
(
    top_category_df
    .write.mode("overwrite")
    .json(f"{output_base}/top_category_json")
)

print("Wrote results under:", output_base)


Wrote results under: /content/output/retail_analytics
